In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="Udacity-Project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-136303
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-136303


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "comp-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 200, 500, 1000),
        '--max_iter' : choice(50, 100, 200, 300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./", 
            compute_target=compute_target,
            vm_size = 'STANDARD_D2_V2',
            entry_script = "train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a5afd688-8611-40dc-81dc-8f52423a1adf
Web View: https://ml.azure.com/experiments/Udacity-Project/runs/HD_a5afd688-8611-40dc-81dc-8f52423a1adf?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-136303/workspaces/quick-starts-ws-136303

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-27T21:19:45.090343][API][INFO]Experiment created<END>\n""<START>[2021-01-27T21:19:45.629139][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-27T21:19:45.868337][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-27T21:19:46.8761883Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_a5afd688-8611-40dc-81dc-8f52423a1adf
Web View: https://ml.azure.com/experiments/Udacity-Project/runs/HD_a5afd688-8611-40dc-81dc-8f52423a1adf?wsid=/subscriptions/9a751

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("Best run metrics:", best_run.get_metrics())

Best run metrics: {'Regularization Strength:': 100.0, 'Max iterations:': 200, 'Accuracy': 0.9176024279210926}


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= ds,
    label_column_name= 'y',
    enable_onnx_compatible_models=True,
    n_cross_validations=3)

In [12]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()

Running on remote.


{'runId': 'AutoML_3ec91f6f-8316-49da-b7d1-918e21214a0d',
 'target': 'comp-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-27T21:36:33.401177Z',
 'endTimeUtc': '2021-01-27T22:20:30.633766Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'comp-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Udacity-Project","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-136303","workspace_name":"quick-starts-ws-136303","region":"southcentralus","compute_target":"comp-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":nu

In [13]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: Udacity-Project,
Id: AutoML_3ec91f6f-8316-49da-b7d1-918e21214a0d_26,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.2442105263157895,
                                                                                                    min_weight_fraction_leaf=

In [14]:
best_run.get_metrics()

{'precision_score_macro': 0.8010831399811714,
 'log_loss': 0.1952991038415797,
 'average_precision_score_weighted': 0.9562975144100898,
 'weighted_accuracy': 0.9585820116658108,
 'matthews_correlation': 0.54469528635616,
 'f1_score_macro': 0.769690039570321,
 'f1_score_micro': 0.9163884276163009,
 'recall_score_weighted': 0.9163884276163009,
 'average_precision_score_micro': 0.9817223931011854,
 'precision_score_micro': 0.9163884276163009,
 'f1_score_weighted': 0.9123067110054194,
 'precision_score_weighted': 0.9103507124586575,
 'recall_score_micro': 0.9163884276163009,
 'AUC_macro': 0.9483659335770392,
 'balanced_accuracy': 0.7466211530790532,
 'norm_macro_recall': 0.4932423061581063,
 'average_precision_score_macro': 0.8283356323691363,
 'accuracy': 0.9163884276163009,
 'AUC_micro': 0.980976659064556,
 'AUC_weighted': 0.9483659335770392,
 'recall_score_macro': 0.7466211530790532,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_3ec91f6f-8316-49da-b7d1-918e21214a0d_26/a